# 1. 读取node和line文件，计算node的加权weight

In [2]:
import numpy as np
import pandas as pd
import math

In [ ]:
def writeNetWeight(id):
    def getWeight(x):
        weight = dataLine[dataLine['Source']==x[0]]['weight'].sum() + dataLine[dataLine['Target']==x[0]]['weight'].sum()
        count = dataLine[dataLine['Source']==x[0]]['weight'].count() + dataLine[dataLine['Target']==x[0]]['weight'].count()
        if count == 0:
            return 0.0
        else:
            return weight
    dataNode = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\' + str(id) + '_node.csv')
    dataLine = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\' + str(id) + '_line.csv')
    dataNode['weightDegree'] = dataNode[['Id']].apply(getWeight, axis=1)
    dataNode.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\' + str(id) + '_nodeWeight.csv', index=False)

In [ ]:
# 获得targetID名单
dataAppID = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv')
appIDs = set(dataAppID['appID'])
for id in appIDs:
    writeNetWeight(id)

# 2.读取nodeWeight，合并人格

In [ ]:
# 读取user_apps
readUserApps = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_apps.csv', names=['userID', 'appList'])
dict_userApps = {}
def getDictUserApps(x):
     dict_userApps[x[0]] = x[1][1:-1].split(', ')
readUserApps.apply(getDictUserApps, axis=1)
readUserApps.head()

In [ ]:
def getSingle(id):
    dataNodeWeight = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\' + str(id) + '_nodeWeight.csv')
    thresholdHead = dataNodeWeight['weightDegree'].max()*0.9
    thresholdTail = dataNodeWeight['weightDegree'].max()*0.2
    dataNodeWeight = dataNodeWeight[(dataNodeWeight['weightDegree']>thresholdTail) & (dataNodeWeight['weightDegree']<thresholdHead)]
    # 合并人格
    global dict1
    global dict0
    dict1 = {}
    dict0 = {}
    def getDict1(x):
        global dict1
        appList = dict_userApps[x[0]]
        for app in appList:
            if dict1.has_key(app):
                dict1[app] = dict1[app]+1
            else:
                dict1[app] = 1
    def getDict0(x):
        global dict0
        appList = dict_userApps[x[0]]
        for app in appList:
            if dict0.has_key(app):
                dict0[app] = dict0[app]+1
            else:
                dict0[app] = 1
    single1 = dataNodeWeight[dataNodeWeight['partition']==1][['Id']]
    single0 = dataNodeWeight[dataNodeWeight['partition']==0][['Id']]
    single1.apply(getDict1, axis=1)
    single0.apply(getDict0, axis=1)
    # dict归一化
    for key in dict1:
        dict1[key] = math.log(dict1[key]+1)/len(single1) #因为权重是log化了，所以这里数量也log化;+1是避免log1=0的情况
    for key in dict0:
        dict0[key] = math.log(dict0[key]+1)/len(single0)
    # 写入dataframe
    global out
    out.loc[out['appID'].count()] = [id, 1, 1, str(dict1)]
    out.loc[out['appID'].count()] = [id, 1, 0, str(dict0)]

In [ ]:
# 写入csv
out = pd.DataFrame(columns=['appID','single','partition','dict'])
# 获得targetID名单
dataAppID = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv')
appIDs = set(dataAppID['appID'])
for id in appIDs:
    getSingle(id)
# 1是app的出现数量/总代表数量
# 2是log（app的出现数量）/总代表数
out.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_single2.csv', index=False)

In [4]:
temp = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_single2.csv')
temp[temp['appID']==83].head()

,appID,single,partition,dict
66,83.0,1.0,1.0,"{'18558': 0.003850817669777474, '799': 0.00385..."
67,83.0,1.0,0.0,"{'4025': 0.003960841031771116, '9671': 0.00396..."
